In [69]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_column",100)
pd.set_option("expand_frame_repr",False)
%matplotlib inline

In [70]:
#1--读取数据
test_data=pd.read_csv(r'D:\文件\学习\数据集\天猫用户复购\data_format1\test_format1.csv')
train_data=pd.read_csv(r'D:\文件\学习\数据集\天猫用户复购\data_format1\train_format1.csv')
user_info=pd.read_csv(r'D:\文件\学习\数据集\天猫用户复购\data_format1\user_info_format1.csv')
user_log=pd.read_csv(r'D:\文件\学习\数据集\天猫用户复购\data_format1\user_log_format1.csv')

In [71]:
import gc
from collections import Counter
import copy

In [72]:
def reduce_mem_usage(df,verbose=True):
    start_mem=df.memory_usage().sum()/1024**2
    number=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns:
        col_type=df[col].dtype
        if col_type in number:
            c_min=df[col].min()
            c_max=df[col].max()
            if str(col_type)[:3]=="int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col]=df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col]=df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col]=df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col]=df[col].astype(np.int64)
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col]=df[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max <np.finfo(np.finfo32).max:
                df[col]=df[col].astype(np.float32)
            elif c_min > np.finfo(np.float64).min and c_max < np.finfo(np.finfo64).max:
                df[col]=df[col].astype(np.float64)
    end_mem=df.memory_usage().sum()/1024**2
    print("压缩前:",start_mem)
    print("压缩后：",end_mem)
    return df

In [73]:
train_data=reduce_mem_usage(train_data)
test_data=reduce_mem_usage(test_data)
user_info=reduce_mem_usage(user_info)
user_log=reduce_mem_usage(user_log)

压缩前: 5.9708251953125
压缩后： 1.7415771484375
压缩前: 5.984855651855469
压缩后： 3.4912166595458984
压缩前: 9.708602905273438
压缩后： 8.090522766113281
压缩前: 2933.3292083740234
压缩后： 1204.7602825164795


In [74]:
user_log=user_log.sort_values(["user_id","time_stamp"])

In [75]:
all_data=train_data.append(test_data)

In [76]:
all_data=all_data.merge(user_info,on="user_id",how="left")
all_data.head(1)

,user_id,merchant_id,label,prob,age_range,gender
0,34176,3906,0.0,NaN,6.0,0.0


In [77]:
list_join_func=lambda x:" ".join([str(i) for i in x])

In [78]:
agg_dict={
    "item_id":list_join_func,
    "cat_id":list_join_func,
    "seller_id":list_join_func,
    "brand_id":list_join_func,
    "time_stamp":list_join_func,
    "action_type":list_join_func
}

In [79]:
rename_dict={
    "item_id":"item_path",
    "cat_id":"cat_path",
    "seller_id":"seller_path",
    "brand_id":"brand_path",
    "time_stamp":"time_stamp_path",
    "action_type":"action_type_path"
}

In [80]:
user_log_path=user_log.groupby("user_id").agg(agg_dict).reset_index().rename(columns=rename_dict)
all_data_path.head(1)

,user_id,merchant_id,label,prob,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
0,34176,3906,0.0,NaN,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...


In [81]:
user_log_path=user_log_path.drop_duplicates("user_id")
user_log_path.head(1)

,user_id,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
0,1,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4750.0 4750.0 4750.0 4750.0 4750.0 7402.0 7402...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...


In [82]:
all_data_path=all_data.merge(user_log_path,on="user_id")
all_data_path.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...


In [83]:
all_data_path.shape

(522341, 12)

In [84]:
all_data_path=all_data_path.drop_duplicates("user_id")
all_data_path.shape

(424170, 12)

In [85]:
all_data_test=all_data_path.head(2000)

In [86]:
def cnt_(x):
    try:
        return len(x.split(" "))
    except:
        return -1

In [87]:
def nunique_(x):
    try:
        return len(set(x.split(" ")))
    except:
        return -1

In [88]:
def max_(x):
    try:
        return np.max([int(i) for i in x.split(" ")])
    except:
        return -1

In [89]:
def min_(x):
    try:
        return np.min([int(i) for i in x.split(" ")])
    except:
        return -1

In [90]:
def user_cnt(df_data,single_col,name):
    df_data[name]=df_data[single_col].apply(cnt_)
    return df_data

In [91]:
def user_nunique(df_data,single_col,name):
    df_data[name]=df_data[single_col].apply(nunique_)
    return df_data

In [92]:
def user_max(df_data,single_col,name):
    df_data[name]=df_data[single_col].apply(max_)
    return df_data

In [93]:
def user_min(df_data,single_col,name):
    df_data[name]=df_data[single_col].apply(min_)
    return df_data

In [94]:
all_data_test=user_cnt(all_data_test,"seller_path","user_cnt")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,user_cnt
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...,451


In [95]:
all_data_test=user_nunique(all_data_test,"seller_path","seller_nunique")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,user_cnt,seller_nunique
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...,451,109


In [96]:
all_data_test=user_nunique(all_data_test,"cat_path","cat_nunique")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,user_cnt,seller_nunique,cat_nunique
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...,451,109,45


In [97]:
all_data_test=user_nunique(all_data_test,"brand_path","brand_nunique")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,user_cnt,seller_nunique,cat_nunique,brand_nunique
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...,451,109,45,108


In [98]:
all_data_test=user_nunique(all_data_test,"time_stamp_path","time_stamp_nunique")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,user_cnt,seller_nunique,cat_nunique,brand_nunique,time_stamp_nunique
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...,451,109,45,108,47


In [99]:
all_data_test=user_nunique(all_data_test,"action_type_path","action_type_nunique")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,user_cnt,seller_nunique,cat_nunique,brand_nunique,time_stamp_nunique,action_type_nunique
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...,451,109,45,108,47,3


In [102]:
all_data_test.index=range(len(all_data_test))

In [103]:
action_test=all_data_test["action_type_path"]
action_test

0       0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
1       0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 ...
2       0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3       0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 ...
4       0 0 2 0 2 2 0 0 0 0 0 3 3 0 0 0 2 0 0 0 0 0 0 ...
                              ...                        
1995    0 0 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 2 0 ...
1996      0 0 3 0 2 0 0 0 0 2 0 0 0 2 2 2 0 0 0 0 1 0 0 0
1997                                    0 0 0 0 0 0 2 0 0
1998    2 2 2 2 0 0 0 0 0 0 0 0 0 2 0 3 3 0 0 0 0 0 3 ...
1999    0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
Name: action_type_path, Length: 2000, dtype: object

In [104]:
action_0=[]
action_1=[]
action_2=[]
action_3=[]
for col in action_test:
    action0=[]
    action1=[]
    action2=[]
    action3=[]
    for i in col:
        if i=="0":
            action0.append(i)
        elif i=="1":
            action1.append(i)
        elif i=="2":
            action2.append(i)
        elif i=="3":
            action3.append(i)
    action_0.append(len(action0))
    action_1.append(len(action1))
    action_2.append(len(action2))
    action_3.append(len(action3))


In [106]:
action_0=pd.DataFrame(action_0)
action_1=pd.DataFrame(action_1)
action_2=pd.DataFrame(action_2)
action_3=pd.DataFrame(action_3)

In [107]:
action_0.columns=["action_0"]
action_1.columns=["action_1"]
action_2.columns=["action_2"]
action_3.columns=["action_3"]

In [108]:
all_data_test=pd.concat([all_data_test,action_0,action_1,action_2,action_3],axis=1)

In [110]:
all_data_test.head()

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,user_cnt,seller_nunique,cat_nunique,brand_nunique,time_stamp_nunique,action_type_nunique,action_0,action_1,action_2,action_3
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...,451,109,45,108,47,3,410,0,34,7
1,230784,4818,0.0,NaN,0.0,0.0,191923 191923 191923 191923 964906 229470 2294...,1023 1023 1023 1023 662 664 664 1544 664 662 6...,3545 3545 3545 3545 4566 2537 2537 2420 2537 4...,5860.0 5860.0 5860.0 5860.0 6322.0 6066.0 6066...,601 601 601 601 614 614 614 614 614 614 618 61...,0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 ...,54,20,17,19,16,2,47,0,7,0
2,362112,2618,0.0,NaN,4.0,1.0,40772 40772 40772 537972 638737 659198 224117 ...,1188 1188 1188 1188 1188 1188 1188 1188 1021 1...,863 863 863 863 863 863 863 863 863 863 1595 8...,5820.0 5820.0 5820.0 6750.0 2229.0 5820.0 6750...,522 522 522 522 522 522 522 522 526 526 526 52...,0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,89,17,13,20,12,2,86,0,3,0
3,34944,2051,0.0,NaN,5.0,0.0,930354 947376 930354 645776 557207 947376 6809...,1252 1252 1252 1252 1252 1252 1252 1252 1252 1...,4893 4893 4893 4893 4893 4893 3172 4893 4893 4...,8031.0 8031.0 8031.0 8031.0 8031.0 8031.0 8031...,522 522 522 522 522 522 522 522 522 522 522 52...,0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 ...,33,10,7,9,6,2,30,0,3,0
4,231552,3828,1.0,NaN,5.0,0.0,13653 631714 631714 419092 450405 419092 87382...,490 1271 1271 1252 1197 1252 656 1252 1252 119...,1922 3955 3955 3647 3647 3647 4257 3647 3647 3...,3844.0 1422.0 1422.0 3019.0 3019.0 3019.0 6742...,531 606 606 1019 1019 1019 1019 1019 1019 1019...,0 0 2 0 2 2 0 0 0 0 0 3 3 0 0 0 2 0 0 0 0 0 0 ...,155,12,25,14,7,3,141,0,12,2
